In [23]:
import numpy as np
import sympy as sp
import scipy as scp
import scipy.stats as sts
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats as sts
import scipy.optimize as opt
from mpl_toolkits import mplot3d

In [24]:
data_prices=np.loadtxt('C:\\Users\\Takan\\Desktop\\Chicago_MAPSS\\Core Study\\Quarter2\\Structural Estimation\\Project_Presentation\\logprices.txt')

# Calculation of Data Moments

In [25]:
data_average_price=np.mean(data_prices) #average log price
print('The average log-price is=',data_average_price)
##############################################################################
data_returns=np.zeros((len(data_prices)-1)) #Calculate the returns from the empirical data
for i in range(0,len(data_prices)-1):
    data_returns[i]=data_prices[i+1]-data_prices[i]

##############################################################################

data_kurtosis=sts.kurtosis(data_returns) # data kurtosis
print('data_kurtosis=',data_kurtosis)
##############################################################################
data_mean_abs_ret=np.mean(np.abs(data_returns)) # mean of absolute returns
print('Average Absolute Returns=',data_mean_abs_ret)
##############################################################################
def autocorr_fun(xvals,lag): # Define autocorrelation function 
    xvals1 = xvals[:(len(xvals)-lag)]
    xvals2 = xvals[lag:]
    sum_product = np.sum((xvals1-np.mean(xvals1))*(xvals2-np.mean(xvals2)))
    return sum_product / ((len(xvals) - lag) * np.std(xvals1) * np.std(xvals2))

data_autocorr_ret1=autocorr_fun(data_returns,1) # first order autocorr of returns
print('First order Autocorr of Returns=',data_autocorr_ret1)
##############################################################################
data_autocorr1=autocorr_fun(np.abs(data_returns),1) # autocorrelations of absolute returns to capture long memory effect
data_autocorr7=autocorr_fun(np.abs(data_returns),7)
data_autocorr10=autocorr_fun(np.abs(data_returns),10)
data_autocorr25=autocorr_fun(np.abs(data_returns),25)
data_autocorr50=autocorr_fun(np.abs(data_returns),50)
data_autocorr100=autocorr_fun(np.abs(data_returns),100)
data_autocorr200=autocorr_fun(np.abs(data_returns),200)
##############################################################################
print('Autocorrel of Abs Ret=',data_autocorr1,data_autocorr7,data_autocorr10,
      data_autocorr25,data_autocorr50,data_autocorr100,data_autocorr200)   

The average log-price is= 6.47989444432
data_kurtosis= 26.93297234893738
Average Absolute Returns= 0.00739804449745
First order Autocorr of Returns= -0.0265566666011
Autocorrel of Abs Ret= 0.227810544662 0.240651472317 0.22581036083 0.175541416824 0.138314837933 0.0998260197641 0.0477729698108


In [26]:
def bootstrap_resample(X, n=None):  # resample with redraw. X is the size of the new sample n is the number of the generated samples
    """ Bootstrap resample an array_like
    Parameters
    ----------
    X : array_like
      data to resample
    n : int, optional
      length of resampled array, equal to len(X) if n==None
    Results
    -------
    returns X_resamples
    """
    if n == None:
        n = len(X)
        
    resample_i = np.floor(np.random.rand(n)*len(X)).astype(int)
    X_resample = X[resample_i]
    return X_resample

In [27]:
boot_ret=np.zeros((5000,1000))  # I generate 1000 new samples with length 5000 from the original series
for j in range(0,1000):
    boot_ret[:,j]=bootstrap_resample(data_returns,n=5000)
##############################################################################
boot_average_abs_ret=np.mean(np.mean(np.abs(boot_ret),axis=0)) # bootstrapped average absolute return
print('The average bootrapped absolute return is=',boot_average_abs_ret)
##############################################################################
boot_data_kurt=np.zeros((1000)) #bootstrapped kurtosis
for j in range(0,1000):
    boot_data_kurt[j]=sts.kurtosis(boot_ret[:,j])
##############################################################################
boot_data_kurt
boot_data_kurt=np.mean(boot_data_kurt)
print('The Bootstrapped Kurtosis of The returns is=',boot_data_kurt)

The average bootrapped absolute return is= 0.00739719004026
The Bootstrapped Kurtosis of The returns is= 24.2365045564


In [28]:
def column_to_vector(matrix, i):
    return [row[i] for row in matrix]
import numpy
def remove_NaN_columns(matrix):
    import scipy
    import math
    from numpy import column_stack, vstack

    columns = matrix.shape[1]
    #print("columns", columns)
    result = []
    skip_column = True
    for column in range(0, columns):
        vector = column_to_vector(matrix, column)
        skip_column = False
        for value in vector:
            # print(column, vector, value, math.isnan(value) )
            if math.isnan(value):
                skip_column = True
        if skip_column == False:
            result.append(vector)
    return column_stack(result)

In [29]:
#dimensions of simulations
length=2000  # number of simulated days
width=100   # number of realizations


# Create epsilons_f for the fundamentslists
unif_vals_f = sts.uniform.rvs(0, 1, size=(length,width))
def norm_draws_f(sigma_f): # random draws for the epsilons N(0,sigma)
    norm_draws = sts.norm.ppf(unif_vals_f, loc=0, scale=sigma_f)
    return norm_draws



# Create epsilons for the chartists
unif_vals_c = sts.uniform.rvs(0, 1, size=(length,width))
def norm_draws_c(sigma_c): # random draws for the epsilons N(0,sigma)
    norm_draws = sts.norm.ppf(unif_vals_c, loc=0, scale=sigma_c)
    return norm_draws



def alpha_price_3(phi,chi,sigma_f,sigma_c): # I keep a_star=-0.15 a_n=2.35 a_p=11.40 mu=0.001
    
    a_star=-0.15
    a_n=1.35
    a_p=11.40
    mu=0.001
    
    
    eps_f=np.zeros((length,width))
    eps_c=np.zeros((length,width))
    
    eps_f=norm_draws_f(sigma_f)
    eps_c=norm_draws_c(sigma_c)
    
    price=np.zeros((length,width))
    price[0,:]=0
    #price[1,:]=0.001
    #price[1,:]=0.001
    a=np.zeros((length,width))
    
    
    def g(z):
        p=z[0]
        g=0
        g=a[0,0]-a_star-a_n*(1/(1+np.exp(-a[0,0]))-1/(1+np.exp(a[0,0])))-a_p*(p)**2
        return g
    sol_1=scp.optimize.root(g,(-a_star/a_p)**0.5)
    price[0,:]=sol_1.x
    #print(sol_1.qtf)
    
    def k(z):
        p=z[0]
        k=0
        k=a[1,0]-a_star-a_n*(1/(1+np.exp(-a[1,0]))-1/(1+np.exp(a[1,0])))-a_p*(p)**2
        return k
    sol_2=scp.optimize.root(g,(-a_star/a_p)**0.5)
    price[1,:]=sol_2.x
    #print(sol_2.qtf)
    
    
    guess=np.zeros((length,width))
    for i in range(2,length):
        for j in range(0,width):
            def f(alpha):
                f=alpha-a_star-a_n*(1/(1+np.exp(-alpha))-1/(1+np.exp(alpha)))-a_p*(price[i-1,j])**2
                return f
            def jac(alpha):
                jac=1-a_n*(1+np.exp(-alpha))**(-2)*np.exp(-alpha)-a_n*(1+np.exp(alpha))**(-2)*np.exp(alpha)
                return jac
            guess[i,j]=a[i-1,j]+(a_p*2*price[i-1,j]/(1-a_n*(1+np.exp(-a[i-1,j]))**(-2)*np.exp(-a[i-1,j])-a_n*(1+np.exp(a[i-1,j]))**(-2)*np.exp(a[i-1,j])))*(price[i-1,j]-price[i-2,j])
            sol= scp.optimize.root(f, (guess[i,j]),tol=1e-10)
            a[i,j]=sol.x
            #print(sol.qtf)
            #print(sol[1].get("qtf"))
            #print(sol[1].get("mesg"))
            price[i,j]=price[i-1,j]+mu*(phi*(-price[i-1,j])/(1+np.exp(-a[i-1,j]))+chi*((price[i-1,j]-price[i-2,j])/(1+np.exp(a[i-1,j])))+(1/(1+np.exp(-a[i-1,j])))*eps_f[i-1,j]+(1/1+np.exp(a[i-1,j]))*eps_c[i-1,j])
            
     
    
    price=remove_NaN_columns(price)
    a=remove_NaN_columns(a)
    
    return (a,price)

In [ ]:
#Test parameter values
a,price=alpha_price_3(1.4175674,5.38736297, 0.87759283,  3.95696199)

In [ ]:
#Plot the alpha series
%matplotlib notebook
plt.plot(a)
plt.show()

In [ ]:
#Plot the price series
%matplotlib notebook
plt.plot(price)
plt.axhline(y=0, color='r', linestyle='-')
plt.gca().legend(('Asset Price','Fair Price'))
plt.show()

In [ ]:
#Plot the fundamentalists shares
n_fund=np.zeros((length,len(price[0])))
for i in range(0,length):
    for j in range(0,len(price[0])):
        n_fund[i,j]=1/(1+np.exp(-a[i,j]))
        
n_chart=np.zeros((length,len(price[0])))
for i in range(0,length):
    for j in range(0,len(price[0])):
        n_chart[i,j]=1/(1+np.exp(a[i,j]))

%matplotlib notebook
plt.plot(n_fund[:,50],linewidth=0.9)
plt.show()

In [ ]:
#Plot the returns series
returns=np.zeros((length,len(price[0])))
for i in range(0,length-1):
    for j in range(0,len(price[0])):
        returns[i,j]=price[i+1,j]-price[i,j]

    
        
        
%matplotlib notebook
plt.plot(returns[:,50],linewidth=0.9)
plt.show()

In [30]:
def err_vec(phi,chi,sigma_f,sigma_c,simple):
    
    data_moms=np.array([[data_mean_abs_ret],[data_kurtosis],
                        [data_autocorr_ret1],[data_autocorr1],[data_autocorr7],[data_autocorr10],
                        [data_autocorr25],[data_autocorr50]])
    
    #####################################   Model Moments ###############################################################
    model_alpha,model_price=alpha_price_3(phi,chi,sigma_f,sigma_c) # calculation of prices and attractivene
    #print(model_price.shape)
    
    #print('model_alpha=',model_alpha)
    #print('model_price=',model_price)
    
    n_fund=np.zeros((2000,len(model_price[0])))
    for i in range(0,2000):
        for j in range(0,len(model_price[0])):
            n_fund[i,j]=1/(1+np.exp(-model_alpha[i,j]))
            
    n_chart=np.zeros((2000,len(model_price[0])))
    for i in range(0,2000):
        for j in range(0,len(model_price[0])):
            n_chart[i,j]=1/(1+np.exp(model_alpha[i,j]))
    
    model_returns=np.zeros((2000-1,len(model_price[0])))
    for i in range(0,2000-1):
        for j in range(0,len(model_price[0])):
            model_returns[i,j]=model_price[i+1,j]-model_price[i,j]

    #print('model_returns=',model_returns)
    
    model_mean_abs_ret=np.mean(np.mean(np.abs(model_returns),axis=0)) #model average absolute return
    #print('model_mean_abs_ret=',model_mean_abs_ret)
    model_kurtosis=np.mean(sts.kurtosis(model_returns,axis=0)) #model kurtosis
    #print('model_kurtosis=',model_kurtosis)
    
    model_autocorr_returns1=np.zeros((len(model_price[0])))
    for j in range(0,len(model_price[0])):
        model_autocorr_returns1[j]=autocorr_fun(model_returns[:,j],1)
    model_autocorr_returns1=np.mean(model_autocorr_returns1) # model 1st order autocorrelation of raw returns
    #print('model_autocorr_returns1=',model_autocorr_returns1)
    
    ################        Calculation of autocorrelations of absolute returns   ###################################
    model_abs_ret=np.abs(model_returns)
    
    model_autocorr_abs_ret1=np.zeros((len(model_price[0])))
    for j in range(0,len(model_price[0])):
        model_autocorr_abs_ret1[j]=autocorr_fun(model_abs_ret[:,j],1)
    model_autocorr_abs_ret1=np.mean(model_autocorr_abs_ret1)
        
    model_autocorr_abs_ret7=np.zeros((len(model_price[0])))
    for j in range(0,len(model_price[0])):
        model_autocorr_abs_ret7[j]=autocorr_fun(model_abs_ret[:,j],7)
    model_autocorr_abs_ret7=np.mean(model_autocorr_abs_ret7)
    
    model_autocorr_abs_ret10=np.zeros((len(model_price[0])))
    for j in range(0,len(model_price[0])):
        model_autocorr_abs_ret10[j]=autocorr_fun(model_abs_ret[:,j],10)
    model_autocorr_abs_ret10=np.mean(model_autocorr_abs_ret10)
    
    model_autocorr_abs_ret25=np.zeros((len(model_price[0])))
    for j in range(0,len(model_price[0])):
        model_autocorr_abs_ret25[j]=autocorr_fun(model_abs_ret[:,j],25)
    model_autocorr_abs_ret25=np.mean(model_autocorr_abs_ret25)
    
    model_autocorr_abs_ret50=np.zeros((len(model_price[0])))
    for j in range(0,len(model_price[0])):
        model_autocorr_abs_ret50[j]=autocorr_fun(model_abs_ret[:,j],50)
    model_autocorr_abs_ret50=np.mean(model_autocorr_abs_ret50)
    
    #model_autocorr_abs_ret100=np.zeros((len(model_price[0]))) # only if you want to use an additional moment
    #for j in range(0,len(model_price[0])):
        #model_autocorr_abs_ret100[j]=autocorr_fun(model_abs_ret[:,j],100)
    #model_autocorr_abs_ret100=np.mean(model_autocorr_abs_ret100)
    
    
    model_moms=np.array([[ model_mean_abs_ret],[model_kurtosis],[model_autocorr_returns1],
                         [model_autocorr_abs_ret1],[model_autocorr_abs_ret7],
                         [model_autocorr_abs_ret10],[model_autocorr_abs_ret25],
                         [model_autocorr_abs_ret50]])
    
    if simple:
        err_vector = model_moms - data_moms
    else:
        err_vector = (model_moms - data_moms) / data_moms
    
    return err_vector

In [31]:
def crit(params,*args):
    phi,chi,sigma_f,sigma_c= params
    W_hat = args
    err = err_vec(phi,chi,sigma_f,sigma_c,simple=False)
    crit_val = np.dot(np.dot(err.T, W_hat), err) 
    print(crit_val)
    print(params)
    
    return crit_val

In [ ]:
#Optimization

phi_init=0.54
chi_init=5.084
sigma_f_init=0.00949
sigma_c_init=4.157
params_init=np.array([phi_init,chi_init,sigma_f_init,sigma_c_init])
W_hat=np.eye(8)
args=W_hat

results=opt.minimize(crit, params_init, args=(args),method='L-BFGS-B',bounds=((1e-08,10+1e-08),(1e-08,10.38-1e-08)
                                                                           ,(1e-08,8.50-1e-08),
                                                                           (1e-08,4.30-1e-08)),options={'eps':0.0001})
print(results)

In [ ]:
#Obtain variance covariance matrix 
invhess=results.hess_inv.todense()
OffDiagNeg = np.array([[1, -1,-1,-1],[-1, 1,-1,-1],[-1,-1,1,-1],[-1,-1,-1, 1]])
vcv = invhess*OffDiagNeg
vcv